In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-3.2.4-bin-hadoop2.7'

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc=SparkContext.getOrCreate()
spark=SparkSession(sc)

In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [11]:
# Read data from 'grades.txt'
data = spark.read.text(r"C:\Users\ahmad\Desktop\grades.txt").rdd.map(lambda x: x[0])

# Process data to extract roll numbers, subjects, and grades
student_data = data.map(lambda x: x.split())

# Define a function to map grades to GPA
def grade_to_gpa(grade):
    grade_mapping = {'A': 4, 'B': 3, 'C': 2, 'D': 1, 'F': 0}
    return grade_mapping.get(grade)

# Calculate GPA for each student
gpa_data = student_data.map(lambda x: (x[0], x[1], grade_to_gpa(x[2])))

# Display the calculated GPA
gpa_data.collect()






[('L22-2100', 'DB', 1),
 ('K21-1601', 'SE', 0),
 ('I21-1601', 'OS', 0),
 ('K21-1702', 'DS', 3),
 ('L21-1705', 'OS', 4),
 ('L22-2101', 'DB', 1),
 ('K21-1601', 'OS', 0),
 ('L21-1601', 'SE', 0),
 ('L21-1702', 'SE', 3),
 ('L21-1705', 'DB', 4)]

In [12]:
gpa_data = student_data.map(lambda x: (x[0], x[1], grade_to_gpa(x[2])))

# Group by RollNumber and calculate the average GPA
average_gpa_data = gpa_data.groupBy(lambda x: x[0]).mapValues(lambda x: sum(val[2] for val in x) / len(x))
average_gpa_data.collect()

[('L22-2100', 1.0),
 ('K21-1601', 0.0),
 ('I21-1601', 0.0),
 ('K21-1702', 3.0),
 ('L21-1705', 4.0),
 ('L22-2101', 1.0),
 ('L21-1601', 0.0),
 ('L21-1702', 3.0)]

In [13]:
def custom_sort_key(roll_number):
    parts = roll_number.split('-')
    campus_order = {'F': 0, 'I': 1, 'K': 2, 'L': 3, 'P': 4}
    year_order = {'16': 0, '17': 1, '99': 2, '01': 3}
    return (
        campus_order.get(parts[0][0], 99),  # Campus
        year_order.get(parts[0][1:], 99),  # Year
        int(parts[1])  # Last part of the roll number
    )
sorted_gpa_data = gpa_data.sortByKey(keyfunc=custom_sort_key)

# Show the sorted data
sorted_gpa_data.collect()

[('I21-1601', 'OS', 0),
 ('K21-1601', 'SE', 0),
 ('K21-1601', 'OS', 0),
 ('K21-1702', 'DS', 3),
 ('L21-1601', 'SE', 0),
 ('L21-1702', 'SE', 3),
 ('L21-1705', 'OS', 4),
 ('L21-1705', 'DB', 4),
 ('L22-2100', 'DB', 1),
 ('L22-2101', 'DB', 1)]